In [1]:
trait Iter[I,A] {
    def getValue(i: I): Option[A]
    def getNext(i: I): I
}

trait Iterable[R,A] {
    type Itr
    def iterIF: Iter[Itr, A]
    def iter(a: R): Itr
}

defined trait Iter
defined trait Iterable

In [2]:
def sumElements[I](xs: I)(implicit IT: Iter[I,Int]): Int = {
    IT.getValue(xs) match {
        case None => 0
        case Some(n) => n + sumElements(IT.getNext(xs))
    }
}

def printElements[I,A](xs: I)(implicit IT: Iter[I,A]): Any = {
    IT.getValue(xs) match {
        case None => 0
        case Some(n) => {
            println(n)
            printElements(IT.getNext(xs))
        }
    }
}

defined function sumElements
defined function printElements

In [3]:
def sumElements2[R](xs: R)(implicit ITR: Iterable[R,Int]) = {
//     sumElements[ITR.Itr](ITR.iter(xs))(ITR.iterIF)
    sumElements(ITR.iter(xs))(ITR.iterIF)
}
def printElements2[R,A](xs: R)(implicit ITR: Iterable[R,A]) = {
//     printElements[ITR.Itr,A](ITR.iter(xs))(ITR.iterIF)
    printElements(ITR.iter(xs))(ITR.iterIF)
}

defined function sumElements2
defined function printElements2

In [4]:
trait ListIF[L,A] {
    def empty: L
    def head(l: L): Option[A]
    def tail(l: L): L
    def cons(a: A, I: L): L
    def append(l1: L, l2: L): L
}
trait TreeIF[T,A] {
    def empty: T
    def node(a: A, l: T, r: T): T
    def head(t: T): Option[A]
    def left(t: T): T
    def right(r: T): T
}

defined trait ListIF
defined trait TreeIF

In [5]:
def testList[L](implicit LI: ListIF[L,Int], IT: Iter[L,Int]) {
    val l = LI.cons(3, LI.cons(5, LI.cons(2, LI.cons(1, LI.empty))))
    println(sumElements(l))
    printElements(l)
}
def testTree[T](implicit TI: TreeIF[T,Int], ITR: Iterable[T,Int]) {
    val t: T = TI.node(3, TI.node(4, TI.empty, TI.empty), TI.node(2, TI.empty, TI.empty))
    println(sumElements2(t))
    printElements2(t)
}

defined function testList
defined function testTree

In [6]:
implicit def listIter[A]: Iter[List[A],A] = {
    new Iter[List[A],A] {
        def getValue(a: List[A]) = a.headOption
        def getNext(a: List[A]) = a.tail
    }
}

implicit def listIF[A]: ListIF[List[A],A] = {
    new ListIF[List[A],A] {
        def empty: List[A] = Nil
        def head(l: List[A]) = l.headOption
        def tail(l: List[A]) = l.tail
        def cons(a: A, l: List[A]) = a :: l
        def append(l1: List[A], l2: List[A]) = l1 ::: l2
    }
}

defined function listIter
defined function listIF

In [7]:
sealed abstract class MyTree[A]
case class Empty[A]() extends MyTree[A]
case class Node[A](value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A]
implicit def treeIF[A]: TreeIF[MyTree[A],A] = {
    new TreeIF[MyTree[A],A] {
        def empty = Empty()
        def node(a: A, l: MyTree[A], r: MyTree[A]) = Node(a,l,r)
        def head(t: MyTree[A]) = t match { 
            case Empty() => None
            case Node(v,_,_) => Some(v)    
        }
        def left(t: MyTree[A]) = t match {
            case Empty() => t
            case Node(_,lt,_) => lt    
        }
        def right(t: MyTree[A]) = t match {
            case Empty() => t
            case Node(_,_,rt) => rt    
        }  
    }
}

defined class MyTree
defined class Empty
defined class Node
defined function treeIF

In [8]:
def treeIterable[L,A](implicit IF: ListIF[L,A], IT: Iter[L,A]): Iterable[MyTree[A], A] = {
    new Iterable[MyTree[A], A] {
        type Itr = L
        def iter(a: MyTree[A]): L = a match {
            case Empty() => IF.empty
            case Node(v, left, right) => IF.cons(v, IF.append(iter(left), iter(right)))    
        }
        val iterIF = IT
    }
}

defined function treeIterable

In [9]:
implicit def treeIterableList[A] = treeIterable[List[A],A]

defined function treeIterableList

In [10]:
testList[List[Int]]

11
3
5
2
1


In [11]:
testTree[MyTree[Int]]

9
3
4
2


In [12]:
implicit def iterIterable[I,A](implicit IT: Iter[I,A]): Iterable[I,A] = {
    new Iterable[I,A] {
        type Itr = I
        def iter(a: I): I = a
        def iterIF: Iter[I,A] = IT
    }
}
val l= List(3,5,2,1)
sumElements2(l)
printElements2(l)

3
5
2
1


defined function iterIterable
l: List[Int] = List(3, 5, 2, 1)
res11_2: Int = 11
res11_3: Any = 0